# Laboratorio 1

Integrantes grupo 24:

* Franco Fernandez Achilli
* Julieta Anahi López
* Idana Noely Wagner Cabrera


## Setup

In [55]:
# Importaciones
import os
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [56]:
# Acceso a Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [57]:
# Path de acceso a la carpeta con los datasets
path = '/content/drive/MyDrive/Colab Notebooks/UPSO - BIG DATA/Laboratorio 1/Datasets'

archivos = os.listdir(path)
archivos

['Dataset_de_ventas_Septiembre.csv',
 'Dataset_de_ventas_Octubre.csv',
 'Dataset_de_ventas_Noviembre.csv',
 'Dataset_de_ventas_Mayo.csv',
 'Dataset_de_ventas_Marzo.csv',
 'Dataset_de_ventas_Junio.csv',
 'Dataset_de_ventas_Julio.csv',
 'Dataset_de_ventas_Febrero.csv',
 'Dataset_de_ventas_Enero.csv',
 'Dataset_de_ventas_Diciembre.csv',
 'Dataset_de_ventas_Agosto.csv',
 'Dataset_de_ventas_Abril.csv',
 'Dataset_de_ventas_Diciembre.gsheet']

In [58]:
# Definimos una lista con la cual ir recorriendo todos los df mediante un for e ir obteniendo la informaicon a analizar
meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio',
         'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

In [59]:
dataframes = {}

for archivo in archivos:
  if archivo.endswith('.csv'): #Asegurarse de que se consideren archivos csv
    mes = archivo.split('_')[3].split('.')[0] # Obtenemos el nombre del mes sin la extensión
    ruta_completa = os.path.join(path,archivo) # Ruta de acceso completa
    df = pd.read_csv(ruta_completa) # Cargamos el csv en un Dataframe
    df_ventas = df.copy() # Hago copia


    # Transformamos todos los valores no numericos en Nan en las columnas Cantidad Pedida y Precio Unitario
    #El errors= 'coerce' transforma los valores no numericos a NaN'
    df_ventas['Cantidad Pedida'] = pd.to_numeric(df_ventas['Cantidad Pedida'], errors='coerce')
    df_ventas['Precio Unitario'] = pd.to_numeric(df_ventas['Precio Unitario'], errors='coerce')

    # Eliminamos todos los NaN y me quedo solo con los valores numericos
    df_ventas = df_ventas.loc[~df_ventas['Cantidad Pedida'].isna()]
    df_ventas = df_ventas.loc[~df_ventas['Precio Unitario'].isna()]

    df_ventas = df_ventas.dropna()


    # Tranformamos todos los valores numericos al tipo de dato int y float segun corresponda
    df_ventas['Cantidad Pedida'] = df_ventas['Cantidad Pedida'].astype(int)
    df_ventas['Precio Unitario'] = df_ventas['Precio Unitario'].astype(float)


    # Creamos una nueva columna con la fecha de compra y tomo los datos de fecha de pedido y lo tranformo en datetime
    df_ventas['Fecha de Compra'] = pd.to_datetime(df_ventas['Fecha de Pedido'])
    df_ventas = df_ventas.drop('Fecha de Pedido', axis = 1)

    # Agregamos una nueva clave y valor, en donde la clave es el mes y el valor es el DataFrame de dicho mes
    dataframes[mes] = df_ventas

In [60]:
# Realizamos una funcion para extraer distintas caracterisitcas
def extraer_datos(datos):

  #DIA Y HORA

  datos['Hora'] = datos['Fecha de Compra'].dt.hour
  datos['Dia'] = datos['Fecha de Compra'].dt.day
  datos['Mes'] = datos['Fecha de Compra'].dt.month_name()
  datos['Dia de la Semana'] = datos['Fecha de Compra'].dt.dayofweek

  # Mapeamos los valores numéricos a nombres de días de la semana
  dias_semana = {0: 'Lunes', 1: 'Martes', 2: 'Miércoles', 3: 'Jueves', 4: 'Viernes', 5: 'Sábado', 6: 'Domingo'}
  datos['Dia de la Semana'] = datos['Dia de la Semana'].map(dias_semana)

  # TOTAL DE VENTAS

  datos['Total Venta'] = (datos['Cantidad Pedida'] * datos['Precio Unitario']).round(2)

  #UBICACION

  # Separamos la direccion de envio en calle ciudad y PO BOX
  direccion_df = datos['Dirección de Envio'].str.split(', ',expand=True)
  direccion_df.columns = ['Calle','Ciudad','PO BOX']

  # Separamos el PO BOX en Estado y Cod Postal
  direccion_df[['Estado', 'Cod Postal']] = direccion_df['PO BOX'].str.split(' ',expand=True)
  datos = pd.concat([datos,direccion_df],axis=True)

  return datos

In [61]:
#Recorremos todos los dataframes para extraer informacion que se utilizará en el analisis
for mes in meses:
  dataframes[mes] = extraer_datos(dataframes[mes])

In [62]:
#Chequeamos del resultado
dataframes['Junio']

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Dirección de Envio,Fecha de Compra,Hora,Dia,Mes,Dia de la Semana,Total Venta,Calle,Ciudad,PO BOX,Estado,Cod Postal
0,209921,USB-C Charging Cable,1,11.95,"950 Walnut St, Portland, ME 04101",2019-06-23 19:34:00,19,23,June,Domingo,11.95,950 Walnut St,Portland,ME 04101,ME,04101
1,209922,Macbook Pro Laptop,1,1700.00,"80 4th St, San Francisco, CA 94016",2019-06-30 10:05:00,10,30,June,Domingo,1700.00,80 4th St,San Francisco,CA 94016,CA,94016
2,209923,ThinkPad Laptop,1,999.99,"402 Jackson St, Los Angeles, CA 90001",2019-06-24 20:18:00,20,24,June,Lunes,999.99,402 Jackson St,Los Angeles,CA 90001,CA,90001
3,209924,27in FHD Monitor,1,149.99,"560 10th St, Seattle, WA 98101",2019-06-05 10:21:00,10,5,June,Miércoles,149.99,560 10th St,Seattle,WA 98101,WA,98101
4,209925,Bose SoundSport Headphones,1,99.99,"545 2nd St, San Francisco, CA 94016",2019-06-25 18:58:00,18,25,June,Martes,99.99,545 2nd St,San Francisco,CA 94016,CA,94016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13617,222905,AAA Batteries (4-pack),1,2.99,"795 Pine St, Boston, MA 02215",2019-06-07 19:02:00,19,7,June,Viernes,2.99,795 Pine St,Boston,MA 02215,MA,02215
13618,222906,27in FHD Monitor,1,149.99,"495 North St, New York City, NY 10001",2019-06-01 19:29:00,19,1,June,Sábado,149.99,495 North St,New York City,NY 10001,NY,10001
13619,222907,USB-C Charging Cable,1,11.95,"319 Ridge St, San Francisco, CA 94016",2019-06-22 18:57:00,18,22,June,Sábado,11.95,319 Ridge St,San Francisco,CA 94016,CA,94016
13620,222908,USB-C Charging Cable,1,11.95,"916 Main St, San Francisco, CA 94016",2019-06-26 18:35:00,18,26,June,Miércoles,11.95,916 Main St,San Francisco,CA 94016,CA,94016


In [63]:
## Concatenacion de todos los dataframes en uno único
df_anual = pd.concat([
           dataframes['Enero'],
           dataframes['Febrero'],
           dataframes['Marzo'],
           dataframes['Abril'],
           dataframes['Mayo'],
           dataframes['Junio'],
           dataframes['Julio'],
           dataframes['Agosto'],
           dataframes['Septiembre'],
           dataframes['Octubre'],
           dataframes['Noviembre'],
           dataframes['Diciembre']])

In [64]:
# Evitamos que ordene los meses alfabeticamente y que respete el orden de los meses del año
df_anual['Mes'] = pd.Categorical(df_anual['Mes'], categories=[
    'January',
    'February',
    'March', 'April',
    'May',
    'June',
    'July',
    'August',
    'September',
    'October', 'November',
    'December'], ordered=True)

## 1. Comportamiento de las ventas en los distintos meses:
● ¿Cómo variaron las ventas a lo largo de los diferentes meses?

● ¿Hubo algún mes que se destacó en términos de ventas?

● ¿Cuál es el ingreso total generado por mes?

### Ventas segun cantidad pedida



In [65]:
# Agrupamos por mes y cantidad pedida
df_ventas_por_cantidad = df_anual.groupby('Mes')['Cantidad Pedida'].sum()
# Reseteamos el indice
df_ventas_por_cantidad = df_ventas_por_cantidad.reset_index(inplace = False)
df_ventas_por_cantidad

,Mes,Cantidad Pedida
0,January,10903
1,February,13449
2,March,17005
3,April,20558
4,May,18667
5,June,15253
6,July,16072
7,August,13448
8,September,13109
9,October,22703


In [66]:
# Grafico

fig = go.Figure()

# Agregamos al objeto figura un grafico de barra con los valores extraidos previamente
fig.add_trace(go.Bar(
        x= df_ventas_por_cantidad['Mes'],
        y=  df_ventas_por_cantidad['Cantidad Pedida'],
        yaxis="y",
        name="Unidades vendidas",
        marker = dict(color=  df_ventas_por_cantidad['Cantidad Pedida'], colorscale = 'RdYlGn',colorbar = dict(title = "Total de ventas")),
        text =  df_ventas_por_cantidad['Cantidad Pedida'],
        textposition = 'auto'
))

# Configuramos titulos y ejes del grafico
fig.update_layout(
    title="Unidades vendidas por mes",
    xaxis=dict(title="Meses"),
    yaxis=dict(title="Unidades vendidas", side="left"))


# Mostramos el grafico
fig.show()

### Ventas segun ingresos

In [67]:
#Agrupamos por mes y total de ingresos
ventas_por_ingreso = df_anual.groupby('Mes')['Total Venta'].sum()
#Reseteamos el indice
ventas_por_ingreso = ventas_por_ingreso.reset_index(inplace = False)
ventas_por_ingreso

,Mes,Total Venta
0,January,1822256.73
1,February,2202022.42
2,March,2807100.38
3,April,3390670.24
4,May,3152606.75
5,June,2577802.26
6,July,2647775.76
7,August,2244467.88
8,September,2097560.13
9,October,3736726.88


In [68]:
# Grafico

fig = go.Figure()

# Agregamos al objeto figura un grafico de barra con los valores extraidos previamente
fig.add_trace(go.Bar(
        x= ventas_por_ingreso['Mes'],
        y=  ventas_por_ingreso['Total Venta'],
        yaxis="y",
        name="Totales Ventas",
        marker = dict(color= ventas_por_ingreso['Total Venta'], colorscale = 'RdYlGn',colorbar = dict(title = "Total de ventas")),
        text = ventas_por_ingreso['Total Venta'],
        textposition = 'auto'
))

# Configuramos titulos y ejes del grafico
fig.update_layout(
    title="Totales vendidos por mes",
    xaxis=dict(title="Meses"),
    yaxis=dict(title="Ingresos", side="left"))


# Mostramos el grafico
fig.show()

###Analisis cantidades vendidas vs ingresos

In [69]:
# Evitamos que ordene los meses alfabeticamente y que respete el orden de los meses del año
df_ventas_por_cantidad['Mes'] = pd.Categorical(df_ventas_por_cantidad['Mes'], categories=[
    'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'], ordered=True)

# Evitamos que ordene los meses alfabeticamente y que respete el orden de los meses del año
ventas_por_ingreso['Mes'] = pd.Categorical(ventas_por_ingreso['Mes'], categories=[
    'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'], ordered=True)

In [70]:
# Crear el objeto figura
fig = go.Figure()
# Añadimos barras de cantidades
fig.add_trace(go.Bar(
        x = df_ventas_por_cantidad['Mes'],
        y= df_ventas_por_cantidad['Cantidad Pedida'],
        name = 'Unidades Vendidas',
        marker = dict(color= 'Light Blue'),
        yaxis = 'y'
)
)

# Añadimos el trazo de ventas
fig.add_trace(go.Scatter(
        x = ventas_por_ingreso['Mes'],
        y= ventas_por_ingreso['Total Venta'],
        name = 'Ingresos',
        line = dict(color= 'Red', shape = 'spline', smoothing = 0.5, width = 3 ),
        yaxis = 'y2'

        )
)

# Configuramos y personalizamos el grafico
fig.update_layout(
    title = 'Ventas por cantidad y por ingresos',
    xaxis = dict(title ='Meses'),
    yaxis = dict(title ='Cantidad', side = 'left'),
    yaxis2= dict(title = 'Ingresos', overlaying = 'y', side = 'right'),
    yaxis_range = [0, max(df_ventas_por_cantidad['Cantidad Pedida'])*1.1],
    yaxis2_range = [0, max(ventas_por_ingreso['Total Venta'])*1.1]

)

##2. Optimización de la publicidad y patrón de ventas por hora:

● ¿Cuál es el momento ideal del día para presentar la publicidad y aumentar la probabilidad de compra?

● ¿Cómo cambian los patrones de ventas por hora a lo largo del año?

● ¿Hay modificaciones en los patrones de ventas durante las horas de mayor actividad en los distintos meses?

### Ventas por hora

In [71]:
#Agrupamos por mes y hora, y por cantidad pedida
ventas_por_hora_cantidad = df_anual.groupby(['Mes', 'Hora'])['Cantidad Pedida'].sum()
ventas_por_hora_cantidad = ventas_por_hora_cantidad.reset_index(inplace = False)
ventas_por_hora_cantidad

,Mes,Hora,Cantidad Pedida
0,January,0,252
1,January,1,137
2,January,2,95
3,January,3,41
4,January,4,53
...,...,...,...
283,December,19,2007
284,December,20,1834
285,December,21,1589
286,December,22,1329


In [72]:
# Grafico
fig = go.Figure()
fig = px.line(ventas_por_hora_cantidad,
              x= 'Hora',
              y= 'Cantidad Pedida',
              title='Cantidad de productos vendidos por hora',
              color = 'Mes')
# Configuramos y personalizamos el grafico
fig.update_xaxes(
    title_text='Hora',
    tickvals=ventas_por_hora_cantidad['Hora']
)

#Mostramos
fig.show()

##3. Distribución de ventas por ubicación:

● ¿En qué ciudades se han registrado las mayores ventas?

● ¿Cómo se comparan las ventas en diferentes estados o regiones?

● ¿Existe variación en las ventas por estado a lo largo de los meses?

### Ventas por ciudad

In [73]:
#Copiamos
ventas_por_ciudad = df_anual.copy()

#Creamos una nueva columna para identificar ciudades y estados
ventas_por_ciudad['Ciudad_Estado'] = ventas_por_ciudad['Ciudad'] + ' (' + ventas_por_ciudad['Estado'] + ')'

#Agrupamos por mes y ciudad_estado y sumamos por cantidad pedida
ventas_por_ciudad = ventas_por_ciudad.groupby(['Mes', 'Ciudad_Estado'])['Cantidad Pedida'].sum().reset_index()
ventas_por_ciudad = ventas_por_ciudad.reset_index( inplace = False)
ventas_por_ciudad

,index,Mes,Ciudad_Estado,Cantidad Pedida
0,0,January,Atlanta (GA),876
1,1,January,Austin (TX),582
2,2,January,Boston (MA),1174
3,3,January,Dallas (TX),878
4,4,January,Los Angeles (CA),1716
...,...,...,...,...
115,115,December,New York City (NY),3816
116,116,December,Portland (ME),352
117,117,December,Portland (OR),1506
118,118,December,San Francisco (CA),6787


In [74]:
#Graficamos
fig = go.Figure()

fig = px.line(ventas_por_ciudad,
                 x="Ciudad_Estado",
                 y="Cantidad Pedida",
                 color="Mes",
                 markers=True,
                 line_shape = 'spline',
                 title='Cantidad Pedida por Ciudad')

# Configuramos y personalizamos el grafico
fig.update_layout(xaxis_title='Ciudad y Estado',
                  yaxis_title='Cantidad Pedida')

fig.show()

### Ventas por estado y por ciudad, por cantidad

In [75]:
# Agrupamos por ciudad y estado
ventas_por_estado_ciudad_cantidad = df_anual.groupby(['Estado', 'Ciudad'])['Cantidad Pedida'].sum()

#Reseteamos el indice
ventas_por_estado_ciudad_cantidad = ventas_por_estado_ciudad_cantidad.reset_index(inplace = False)
ventas_por_estado_ciudad_cantidad

,Estado,Ciudad,Cantidad Pedida
0,CA,Los Angeles,33289
1,CA,San Francisco,50239
2,GA,Atlanta,16602
3,MA,Boston,22528
4,ME,Portland,2750
5,NY,New York City,27932
6,OR,Portland,11303
7,TX,Austin,11153
8,TX,Dallas,16730
9,WA,Seattle,16553


In [76]:
#Graficamos
fig = px.bar(ventas_por_estado_ciudad_cantidad,
             x="Estado",
             y="Cantidad Pedida",
             color="Ciudad",
             text = "Ciudad",
             title="Cantidad Pedida por Ciudad y Estado")

#Mostramos
fig.show()

## 4. Análisis del producto más vendido:
● ¿Cuál es el producto más vendido en general y en cada mes?

● ¿Qué factores crees que han contribuido al éxito de ese producto en particular?

In [77]:
# Creamos la variable "venta_productos_anuales" donde almacenamos el DataFrame df_anual ajustado para nuestras necesidades.
ventas_productos_anuales = df_anual.groupby("Producto")["Cantidad Pedida"].sum()

# Reiniciamos el índice para luego poder graficarlo
ventas_productos_anuales = ventas_productos_anuales.reset_index(inplace = False)
ventas_productos_anuales

,Producto,Cantidad Pedida
0,20in Monitor,4129
1,27in 4K Gaming Monitor,6244
2,27in FHD Monitor,7550
3,34in Ultrawide Monitor,6199
4,AA Batteries (4-pack),27635
5,AAA Batteries (4-pack),31017
6,Apple Airpods Headphones,15661
7,Bose SoundSport Headphones,13457
8,Flatscreen TV,4819
9,Google Phone,5532


In [78]:
# Graficamos el DataFrame con las ventas anuales de cada producto.
fig = px.bar(
    ventas_productos_anuales,
    x="Producto",
    y="Cantidad Pedida",
    color="Cantidad Pedida",
    title= "Ventas anuales de cada producto")

# Cambiamos el texto para que refleje de mejor manera los datos.
fig.update_layout(
    coloraxis_colorbar=dict(title="Ventas"),
    yaxis = dict(title="Cantidad de ventas")
)

fig.show()

In [79]:
# Generamos una variable con los datos necesarios para realizar el análisis de las ventas mes a mes de cada producto.
df_productos = df_anual.groupby(["Mes", "Producto"])["Cantidad Pedida"].sum()
df_productos = df_productos.reset_index(inplace = False)

In [80]:
#Creamos la figura con los valores correspondientes al caso de estudio.
fig = px.bar(
    df_productos,
    x="Producto",
    y="Cantidad Pedida",
    color="Mes"
)

# Realizamos ajustes en la figura para mejorar la visibilidad de los datos.
fig.update_layout(
    title="Cantidad de ventas por mes de cada producto",
    xaxis=dict(title="Producto"),
    yaxis=dict(title="Unidades vendidas", side="left")
)

fig.show()

## 5. Tendencia ventas:

● ¿Existe alguna tendencia o patrón en las ventas que se repita a lo largo de los días
en los diferentes meses?

● ¿Cómo varían las ventas a lo largo de los diferentes días de la semana?

● ¿Se observa alguna diferencia significativa en las ventas entre los días laborables y
los fines de semana?


In [81]:
# Generamos las variables que nos permitan analizar la cantidad de ventas mensuales.
ventas_por_mes = df_anual.groupby(['Mes', 'Dia'])['Cantidad Pedida'].sum()
ventas_por_mes = ventas_por_mes.reset_index(inplace = False)
ventas_por_mes = ventas_por_mes[ventas_por_mes['Cantidad Pedida'] != 0]

# Creamos un diccionario que contiene los días no laborables de cada mes (Incluye fin de semana y eventos especiales).
dias_no_laborables = {
    "January": [1,5,6,12,13,19,20,21,26,27],
    "February": [2,3,9,10,14,16,17,18,24],
    "March": [2,3,9,10,16,17,23,24,31],
    "April": [6,7,13,14,20,21,27,28],
    "May": [4,5,11,12,18,19,25,26,27],
    "June": [1,2,8,9,15,16,22,23,29,30],
    "July":  [4,6,7,13,14,20,21,27,28],
    "August": [3,4,10,11,17,18,24,25],
    "September": [1,2,7,8,14,15,21,22,28,29],
    "October": [5,6,12,13,14,19,20,26,27,31],
    "November": [2,3,9,10,11,16,17,23,24,28],
    "December": [1,7,8,14,15,21,22,25,28,29]
}

# Creamos una lista vacía para almacenar los resultados
lista_datos_no_laborables = []

# Se itera a través de los meses y días no laborables
for mes, dias in dias_no_laborables.items():
    for dia in dias:
        # Se filtra el DataFrame para obtener las filas que coinciden con el día no laborable actual
        filtro = (ventas_por_mes["Mes"] == mes) & (ventas_por_mes["Dia"] == dia)

        # Se agregan los datos filtrados a la lista
        lista_datos_no_laborables.append(ventas_por_mes[filtro])

# Ahora df_dias_no_laborables contendrá las filas correspondientes a los días no laborables
df_dias_no_laborables = pd.concat(lista_datos_no_laborables, ignore_index=True)
df_dias_no_laborables

,Mes,Dia,Cantidad Pedida
0,January,1,384
1,January,5,355
2,January,6,319
3,January,12,340
4,January,13,368
...,...,...,...
107,December,21,875
108,December,22,874
109,December,25,902
110,December,28,931


In [82]:
#Agregamos una columna para modificar los datos de la leyenda
df_dias_no_laborables['Mes_Leyenda'] = df_dias_no_laborables['Mes'].astype(str) + ' No laborable'

# Crear una figura de Plotly Express para graficar los dias no laborables
fig = px.scatter(
    df_dias_no_laborables,
    x='Dia',
    y='Cantidad Pedida',
    color='Mes_Leyenda',
    color_discrete_sequence=px.colors.qualitative.Set1,  # Cambiamos la paleta de colores
)

fig.update_traces(marker=dict(size=10)) # agrandamos el tamaño de los puntos

# Recorremos el df y graficamos las unidades vendidas por día
for mes, datos_mes in ventas_por_mes.groupby('Mes'):
    line_trace = go.Scatter(
        x=datos_mes['Dia'],
        y=datos_mes['Cantidad Pedida'],
        mode='lines',
        name=mes,  # Usar el nombre del mes como etiqueta
        line=dict(width=3, shape="spline")
    )
    fig.add_trace(line_trace)

# Personalizar el gráfico
fig.update_layout(
    title='Ventas mensuales vs Días no laborables',
    xaxis = dict(tickvals = datos_mes["Dia"]),
    xaxis_title='Día',
    yaxis_title='Unidades vendidas',
    showlegend=True,  # Mostrar leyenda para los meses
)

# Mostrar el gráfico interactivo
fig.show()

## 6. Impacto de eventos especiales en las ventas:
● ¿Se ha observado algún aumento o disminución significativa en las ventas en días cercanos a eventos especiales, como días festivos?

● ¿Qué eventos específicos han tenido un impacto notable en el comportamiento de las ventas y cómo se manifestó ese impacto?

In [83]:
# Creamos la variable dias_festivos con los datos requeridos para el análisis.
dias_festivos = df_anual.groupby(['Mes', 'Dia'])['Cantidad Pedida'].sum()
dias_festivos = dias_festivos.reset_index(inplace = False)

# Mostramos los primeros 5 elementos para verificar.
dias_festivos.head()

,Mes,Dia,Cantidad Pedida
0,January,1,384
1,January,2,368
2,January,3,330
3,January,4,330
4,January,5,355


In [84]:
# Creamos un diccionario que contendrá los días festivos de cada mes.
dias_festivos_por_mes = {
    "January": [1, 21],
    "February": [18],
    "March": [0],
    "April": [0],
    "May": [27],
    "June": [0],
    "July": [4],
    "August": [0],
    "September": [2],
    "October": [14],
    "November": [11,28],
    "December": [25]

}

# Crea una nueva columna "Festivo" en False por defecto
dias_festivos['Festivo'] = False

# Define una función para marcar los días festivos en función del mes
def marcar_festivos(row):
    mes = row['Mes']
    dia = row['Dia']
    if mes in dias_festivos_por_mes and dia in dias_festivos_por_mes[mes]:
        return True
    else:
        return False

# Aplica la función a cada fila del DataFrame para marcar los días festivos
dias_festivos['Festivo'] = dias_festivos.apply(marcar_festivos, axis=1)

In [85]:
# Eliminamos las filas que cumplan con la condición ["Cantidad Pedida"] == 0.
dias_festivos
dias_festivos = dias_festivos.drop(dias_festivos[(dias_festivos['Cantidad Pedida'] == 0)].index)

In [97]:
# Supongamos que ya tienes un DataFrame df_anual con las columnas mencionadas

# Crear una figura de Plotly Express
fig = px.scatter(
    dias_festivos,
    x='Dia',
    y='Cantidad Pedida',
    color='Mes',
    color_discrete_sequence=px.colors.qualitative.Set1,  # Colores para los meses
    symbol='Festivo',
    symbol_map={False: 'circle', True: 'x'},  # Usar círculos para no festivos y 'x' para festivos
)

# Personalizar el tamaño de los símbolos para festivos
fig.update_traces(marker=dict(size=12), selector=dict(marker_symbol='x'))

# Personalizar el gráfico
fig.update_layout(
    title='Cantidad Pedida por Día',
    xaxis_title='Día',
    yaxis_title='Cantidad Pedida',
    showlegend=True,  # Mostrar leyenda para los meses
)

# Mostrar el gráfico interactivo
fig.show()

## Pregunta Adicional


---

Además de las preguntas mencionadas anteriormente, te invito a pensar en una pregunta
adicional que podría ser relevante para analizar los datos de ventas mensuales. Esta
pregunta debe estar basada en la información que se proporciona en los conjuntos de
datos. Puede estar relacionada con tendencias, comparaciones, patrones o cualquier otro
aspecto que despierte tu interés. Trata de ser creativo/a y pensar en cómo podrías explorar
aún más estos datos!

¿ Que analisis podemos hacer con respecto a la fidelidad de los clientes? ¿Son siempre compras ocasionales o hay una cartera de cliente regulares?

In [92]:
#Agrupamos por calle y contamos la cantidad de pedidos
fidelidad_clientes = df_anual.groupby(['Calle'])['Cantidad Pedida'].count()
#Reseteamos el indice
fidelidad_clientes = fidelidad_clientes.reset_index(inplace = False)
#Ordemos de mayor a menor cantidad de pedidos
fidelidad_clientes.sort_values(by = "Cantidad Pedida", ascending = False)


,Calle,Cantidad Pedida
28205,640 West St,14
29214,660 Center St,14
34046,753 Cedar St,14
25729,594 6th St,14
46141,986 7th St,14
...,...,...
12578,340 Park St,1
12576,340 Meadow St,1
18915,463 4th St,1
5318,200 Jackson St,1


In [93]:
#Agregamos la columna de categoria de cliente
fidelidad_clientes['Categoria de cliente'] = 0
fidelidad_clientes

,Calle,Cantidad Pedida,Categoria de cliente
0,1 11th St,3,0
1,1 12th St,4,0
2,1 13th St,1,0
3,1 14th St,6,0
4,1 1st St,2,0
...,...,...,...
46832,999 Walnut St,3,0
46833,999 Washington St,1,0
46834,999 West St,4,0
46835,999 Willow St,5,0


In [94]:
#Definimos una funcion que categorice el tipo de cliente en funcion de la cantidad de pedidos en el año
def categoria_cliente(row):
  compras = row['Cantidad Pedida']
  if compras == 1:
    return"Unica compra"
  elif compras > 1 and compras <=4:
    return "Ocasional"
  elif compras > 4 and compras <=9:
    return"Frecuente"
  elif compras > 9:
    return "Muy Frecuente"

#Aplicamos la funcion a todas las filas
fidelidad_clientes['Categoria de cliente'] = fidelidad_clientes.apply(categoria_cliente, axis=1)
fidelidad_clientes

,Calle,Cantidad Pedida,Categoria de cliente
0,1 11th St,3,Ocasional
1,1 12th St,4,Ocasional
2,1 13th St,1,Unica compra
3,1 14th St,6,Frecuente
4,1 1st St,2,Ocasional
...,...,...,...
46832,999 Walnut St,3,Ocasional
46833,999 Washington St,1,Unica compra
46834,999 West St,4,Ocasional
46835,999 Willow St,5,Frecuente


In [95]:
#  Agrupamos por categoria y contamos cuantos hay de cada una
fidelidad_clientes = fidelidad_clientes.groupby('Categoria de cliente')['Cantidad Pedida'].count()
#Reseteamos el indice
fidelidad_clientes = fidelidad_clientes.reset_index (inplace = False)
fidelidad_clientes

,Categoria de cliente,Cantidad Pedida
0,Frecuente,16228
1,Muy Frecuente,453
2,Ocasional,26013
3,Unica compra,4143


In [96]:
# Datos de ejemplo
categorias = fidelidad_clientes['Categoria de cliente']
cantidad_clientes = fidelidad_clientes['Cantidad Pedida']

# Creamos un gráfico de torta
fig = go.Figure(data=[go.Pie(labels=categorias, values=cantidad_clientes)])

# Personaliza el diseño del gráfico
fig.update_traces(
    hoverinfo="label+value",  # Muestra etiquetas y porcentaje en el hover
    textinfo="label+percent",           # Muestra el valor en las secciones del gráfico
    pull=[0, 0.1, 0.1, 0.1],    # Separa una sección para resaltar
)

fig.update_layout(
    title="Categorización de Clientes por Cantidad de Compras",
    legend=dict(title="Categorías"),
)

# Mostramos
fig.show()